# PaddleMD测试
要在命令行测试，首先需要安装相关的库
然后使用develop方式安装PaddleMD
最后执行命令

## 在notebook中安装moleculekit
然后在命令行安装openmm
`conda install openmm -c conda-forge -i https://mirror.baidu.com/pypi/simple`

6.4日测试，发现需要使用`conda install openmm -c conda-forge`
且安装的时候需要安装cuda

## 测试记录：
发现测试水分子的时候，很多值都没有变动。是不是没有反向传播的原因？

综合测试的时候，Energy diff: 4.206e-08 Force diff 1.697e+00
能量差值很小，而力场差别很大！ 
比如2watersperiodic： electrostatic Energy diff: 3.203e-06 Force diff: 1.695e+00
静电力场差值很大！ 





# 先安装需要的库
openmm需要在命令行使用conda安装，moleculekit可以使用conda安装，也可以pip安装特定版本：
```
conda install openmm -c conda-forge
conda install moleculekit -c acellera

!pip install moleculekit==1.0.8 -i https://mirror.baidu.com/pypi/simple
```


In [5]:
# !pip install moleculekit==1.0.8 -i https://mirror.baidu.com/pypi/simple

## 安装paddlemd开发模式

In [1]:
!python setup.py develop

fatal: No names found, cannot describe anything.
Could not get version tag. Defaulting to version 0
running develop
running egg_info
writing paddlemd.egg-info/PKG-INFO
writing dependency_links to paddlemd.egg-info/dependency_links.txt
writing requirements to paddlemd.egg-info/requires.txt
writing top-level names to paddlemd.egg-info/top_level.txt
reading manifest file 'paddlemd.egg-info/SOURCES.txt'
writing manifest file 'paddlemd.egg-info/SOURCES.txt'
running build_ext
Creating /opt/conda/lib/python3.6/site-packages/paddlemd.egg-link (link to .)
Adding paddlemd 0 to easy-install.pth file

Installed /code/6paper/PaddleMD
Processing dependencies for paddlemd==0
error: pandas 0.24.2 is installed but pandas>=1.1.2 is required by {'moleculekit'}


## 开始执行集合测试程序
### 解决报错,把requires_grad改成stop_gradient

### 效果尚可
与openmm的差值为
```
Running test: test-data/thrombin-ligand-amber/
  angle Energy diff: -5.433e-05 Force diff: 1.108e-03
  bond Energy diff: 1.861e-04 Force diff: 4.413e-03
  dihedral Energy diff: 3.932e-05 Force diff: 6.289e-04
  lennardjones Energy diff: 1.829e-04 Force diff: 5.169e-04
```
gpu显存占用 4874MiB / 15109MiB

In [ ]:
!python tests/test_paddlemd.py

/opt/conda/lib/python3.6/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
test_paddlemd (__main__._TestPaddleMD) ... 
Running test: test-data/4dihedrals/
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size chan

In [2]:
from moleculekit.molecule import Molecule

mol = Molecule('3ptb')
mol.view()

2022-06-04 22:36:08,935 - moleculekit.readers - INFO - Attempting PDB query for 3ptb
2022-06-04 22:36:12,998 - moleculekit.rcsb - WARNING - Failed to connect to URL https://files.rcsb.org/download/3ptb.pdb with error <urlopen error [Errno -3] Temporary failure in name resolution>. Sleeping 5s and retrying.


# 在命令行测试水分子

执行命令为`./bin/paddlemd --conf tests/water/50000water_conf.yaml `

torch下的速度7分36秒 飞桨下steps: 10大约23分钟
现在1000step大约2分28秒,这样算下来50000step需要125分钟，大约2小时。 目前torchmd比paddlemd快16倍。

In [18]:
# !./bin/paddlemd --conf tests/water/50000water_conf.yaml # 10step 快速测试
!./bin/paddlemd --conf tests/water/water_conf.yaml

/opt/conda/lib/python3.6/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Force terms:  ['LJ', 'Bonds', 'Angles', 'Electrostatics']
W0605 23:04:10.737171 27221 gpu_context.cc:278] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 11.0, Runtime API Version: 10.2
W0605 23:04:10.742141 27221 gpu_context.cc:306] device: 0, cuDNN Version: 7.6.
/opt/conda/lib/python3.6/site-packages/paddle/fluid/dygraph/math_op_patch.py:278: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.bool, the right dtype will convert to paddle.float32
  format(lhs_dtype, rhs_dtype, lhs_dtype))
Writing logs to  mywaterrun/monitor_0.csv
Writing logs to  mywaterrun/monitor_1.csv
100%|███████████████████████████████████████████| 20/20 [02:25<00:00,  7.27s/it]


# 在命令行测试prod alanine dipeptide前丙氨酸二肽
能跑下去，但是估计时间比较多。大约需要3.5小时。
跑出来的数值跟torchmd接近。

In [3]:
!./bin/paddlemd --conf tests/prod_alanine_dipeptide_amber/conf.yaml

/opt/conda/lib/python3.6/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Force terms:  None
2022-06-06 09:12:56,036 - root - WARNING - Warning: importing 'simtk.openmm' is deprecated.  Import 'openmm' instead.
W0606 09:12:58.064165  5913 gpu_context.cc:278] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 11.0, Runtime API Version: 10.2
W0606 09:12:58.067812  5913 gpu_context.cc:306] device: 0, cuDNN Version: 7.6.
/opt/conda/lib/python3.6/site-packages/paddle/fluid/dygraph/math_op_patch.py:278: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.bool, the right dtype will convert to paddle.float32
  format(lhs_dtype, rhs_dtype, lhs_dtype))
Writing logs to  /tmp/alanine_dipeptide/monitor_0.csv
Iter  Epot            fmax    
   0   -394.649102    94.587214
   1   -2

In [ ]:
## 测试结果
飞桨  405   -1362.860203    3.397800
torch 405   -1362.809237    10.436576
1013   -1385.292748    2.422747

# 调试
## 执行测试之后卡住
```
W0503 16:17:49.902498  3748 device_context.cc:465] device: 0, cuDNN Version: 7.6.
/opt/conda/lib/python3.6/site-packages/paddle/fluid/dygraph/math_op_patch.py:253: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.bool, the right dtype will convert to paddle.float32
  format(lhs_dtype, rhs_dtype, lhs_dtype))

Running test: test-data/thrombin-ligand-amber/
```
## 卡住之后，手工中断，报错信息：
```
Running test: test-data/thrombin-ligand-amber/
^C
Traceback (most recent call last):
  File "tests/test_paddlemd.py", line 551, in <module>
    unittest.main(verbosity=2)
  File "/opt/conda/lib/python3.6/unittest/main.py", line 95, in __init__
    self.runTests()
  File "/opt/conda/lib/python3.6/unittest/main.py", line 256, in runTests
    self.result = testRunner.run(self.test)
  File "/opt/conda/lib/python3.6/unittest/runner.py", line 176, in run
    test(result)
  File "/opt/conda/lib/python3.6/unittest/suite.py", line 84, in __call__
    return self.run(*args, **kwds)
  File "/opt/conda/lib/python3.6/unittest/suite.py", line 122, in run
    test(result)
  File "/opt/conda/lib/python3.6/unittest/suite.py", line 84, in __call__
    return self.run(*args, **kwds)
  File "/opt/conda/lib/python3.6/unittest/suite.py", line 122, in run
    test(result)
  File "/opt/conda/lib/python3.6/unittest/case.py", line 653, in __call__
    return self.run(*args, **kwds)
  File "/opt/conda/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "tests/test_paddlemd.py", line 408, in test_paddlemd
    system.pos, system.box, system.forces, returnDetails=True
  File "/code/6paper/PaddleMD/paddlemd/forces.py", line 342, in compute
    forces[i] = paddleindex_add(forces[i], 0, ava_idx[:, 0], -forcevec)
  File "/code/6paper/PaddleMD/paddlemd/forces.py", line 38, in paddleindex_add
    x[index[i]] += source[i]
  File "/opt/conda/lib/python3.6/site-packages/paddle/fluid/dygraph/varbase_patch_methods.py", line 778, in __setitem__
    return _setitem_impl_(self, item, value)
  File "/opt/conda/lib/python3.6/site-packages/paddle/fluid/variable_index.py", line 703, in _setitem_impl_
    inplace_map={"Input": "Out"})
  File "/opt/conda/lib/python3.6/site-packages/paddle/fluid/framework.py", line 3604, in append_op
    inplace_map)
  File "/opt/conda/lib/python3.6/site-packages/paddle/fluid/dygraph/tracer.py", line 309, in trace_op
    not stop_gradient, inplace_map if inplace_map else {})
KeyboardInterrupt
```

## 测试水分子报错
```
  File "/code/6paper/PaddleMD/bin/../paddlemd/run.py", line 76, in setup
    paddle.manual_seed(args.seed)
AttributeError: module 'paddle' has no attribute 'manual_seed'
root@f7d7b1900cfff011ec0b6ec0bb60bea6856f-task1-0:/code/6paper/PaddleMD# 
```
问题是我看原代码，76句那句是注释掉的啊！出了灵异事件了 。
晕，发现是改错文件了。并没有修改paddlemd那个目录的run文件，修改之。

##  [operator < elementwise_add > error]
```
  File "/code/6paper/PaddleMD/paddlemd/wrapper.py", line 19, in paddleindexjia
    tmp[xindex] += y
  File "/opt/conda/lib/python3.6/site-packages/paddle/fluid/dygraph/math_op_patch.py", line 299, in __impl__
    return math_op(self, other_var, 'axis', axis)
ValueError: (InvalidArgument) Broadcast dimension mismatch. Operands could not be broadcast together with the shape of X = [3, 3] and the shape of Y = [2, 3]. Received [3] in X is not equal to [2] in Y at i:0.
  [Hint: Expected x_dims_array[i] == y_dims_array[i] || x_dims_array[i] <= 1 || y_dims_array[i] <= 1 == true, but received x_dims_array[i] == y_dims_array[i] || x_dims_array[i] <= 1 || y_dims_array[i] <= 1:0 != true:1.] (at /paddle/paddle/phi/kernels/funcs/common_shape.h:84)
  [operator < elementwise_add > error]
```
这样应该是paddleeye的锅。在下面处理paddleeye
将所有的eye相关语句，修改成这样
```
# box = box[:, torch.eye(3).bool()]
box = box*paddle.eye(3)
box = box.sum(1)

同时发现在forces.py文件中，有这样一句：
sbox = box[i][torch.eye(3).bool()]
相关的飞桨可以写为：
sbox = box[i][paddle.eye(3).astype(paddle.bool)]

```
脑子有点乱了，系统里面用到的eye特别多。。。

6.3日，回头看这里的报错，明显是paddleindexjia的报错啊！ 因为这个函数当时写的时候维度是写死的，所以碰到新情况shpae就对不齐了。
解决的方法是使用numpy语句解决，加上tensor的来回变换，一共4句：
```
#                 pos[:, group] -= offset.unsqueeze(1)
#                 pos = paddleindexjia(pos, -offset, group)
                pos = pos.numpy()
                offset = offset.unsqueeze(1).numpy()
                pos[:, group] -= offset # 尝试使用numpy来处理 相关语句共4句
                pos = paddle.to_tensor(pos)

#             pos[:, self.nongrouped] -= offset.unsqueeze(1)
#             pos = paddleindexjia(pos, -offset, self.nongrouped)
            pos = pos.numpy()
            offset = offset.unsqueeze(1).numpy()
            pos[:, self.nongrouped] -= offset # 尝试使用numpy来处理 相关语句共4句
            pos = paddle.to_tensor(pos)
```
两个地方需要修改，并且第二部分应该用参数self.nongrouped，以前误写成group，这次一并改回。
## 专门修改eye相关函数
比如这样的，我认为可以提速：
```
        for r in range(box.shape[0]):
            self.box[r][torch.eye(3).bool()] = torch.tensor(
                box[r], dtype=self.box.dtype, device=self.box.device
            )
```


## 处理eye相关语句
torch相关代码为`box = box[:, torch.eye(3).bool()] `

## 修改paddleindex_add
重写了paddleindex_add之后，测试水分子时报错：
```
[291, 3] indices.shape:[291] value.shape:[291, 3]
==arr.shape:[291, 3] indices.shape:[291] value.shape:[291, 3]
==arr.shape:[291, 3] indices.shape:[97] value.shape:[97, 3]
==arr.shape:[291, 3] indices.shape:[97] value.shape:[97, 3]
==arr.shape:[291, 3] indices.shape:[97] value.shape:[97, 3]
==arr.shape:[291, 3] indices.shape:[15168] value.shape:[15168, 3]
```
不明白为什么会出来15168 这么大的值 ...
 ==arr.shape:[291, 3] indices.shape:[15168] value.shape:[15168, 3]
 
 还有这么大的：Running test: test-data/thrombin-ligand-amber/
==force line 276 explicit_forces:True electrostatics len of idx14[:, 0]:12320

index_add里面加入截断。 要去看看torch怎么处理的，估计torch是按照顺序覆盖的。

最终参考torch.index_add的实现，对paddleindex_add进行了相应修改！不截断，全部数据按照索引加上去！ 



## 测试报错
```
======================================================================
ERROR: test_paddlemd (__main__._TestPaddleMD) (system='test-data/4dihedrals/')
----------------------------------------------------------------------
Traceback (most recent call last):
  File "tests/test_paddlemd.py", line 408, in test_paddlemd
    system.pos, system.box, system.forces, returnDetails=True
  File "/code/6paper/PaddleMD/paddlemd/forces.py", line 282, in compute
    spos, self.par.impropers[:, 0:2], sbox
  File "/code/6paper/PaddleMD/paddlemd/forces.py", line 421, in calculate_distances
    dist = paddle.norm(direction_vec, axis=1)
  File "/opt/conda/lib/python3.6/site-packages/paddle/tensor/linalg.py", line 436, in norm
    name=name)
  File "/opt/conda/lib/python3.6/site-packages/paddle/tensor/linalg.py", line 308, in vector_norm
    'keepdim', keepdim, 'asvector', asvector)
ValueError: (InvalidArgument) Attr(axis) value should be in range [-R, R-1], R is the rank of Input(X). But received axis: 1, R: 1. Current Input(X)'s shape is=[3].
  [Hint: Expected axis < x_rank, but received axis:1 >= x_rank:1.] (at /paddle/paddle/phi/infermeta/unary.cc:1488)
  [operator < p_norm > error]
```
 value should be in range [-R, R-1], R is the rank of Input(X). But received axis: 1, R: 1. Current Input(X)'s shape is=[3].

还是函数里面没弄好导致的，全部逻辑理顺，终于顺利测试完成了。（调哪个函数？ 为什么6.5日又出现了这个报错？ ）

 
 ## 测试的时候报错没有openmm
 发现pip安装的openmm不行，要用canda安装
需要使用`conda install openmm -c conda-forge`
且安装的时候会安装cuda

## 在测试时，发现很多时候没有用到paddle.norm部分，用的地方报错
```python
Traceback (most recent call last):
  File "tests/test_paddlemd.py", line 408, in test_paddlemd
    system.pos, system.box, system.forces, returnDetails=True
  File "/code/6paper/PaddleMD/paddlemd/forces.py", line 201, in compute
    spos, self.par.dihedrals[:, 0:2], sbox
  File "/code/6paper/PaddleMD/paddlemd/forces.py", line 422, in calculate_distances
    dist = paddle.norm(direction_vec, axis=1)
  File "/opt/conda/lib/python3.6/site-packages/paddle/tensor/linalg.py", line 436, in norm
    name=name)
  File "/opt/conda/lib/python3.6/site-packages/paddle/tensor/linalg.py", line 308, in vector_norm
    'keepdim', keepdim, 'asvector', asvector)
ValueError: (InvalidArgument) Attr(axis) value should be in range [-R, R-1], R is the rank of Input(X). But received axis: 1, R: 1. Current Input(X)'s shape is=[3].
  [Hint: Expected axis < x_rank, but received axis:1 >= x_rank:1.] (at /paddle/paddle/phi/infermeta/unary.cc:1488)
  [operator < p_norm > error]
重点检查：  
def calculate_distances(atom_pos, atom_idx, box):
#     print(f"==calculate_distances {atom_pos, atom_idx, box}")
#     print(f"==calculate_distances atom_pos, atom_idx, box:{atom_pos.shape, atom_idx.shape, box.shape}")

    direction_vec = wrap_dist(atom_pos[atom_idx[:, 0]] - atom_pos[atom_idx[:, 1]], box)
    print(f"==line 423 of forces direction_vec.shape:{direction_vec.shape}")
    dist = paddle.norm(direction_vec, axis=1)
    direction_unitvec = direction_vec / dist.unsqueeze(1)
    return dist, direction_unitvec, direction_vec
```

## 集中全力处理test_replicas, Attr(axis) value should be in range [-R, R-1]
```python
test_replicas (__main__._TestPaddleMD) ... ERROR

======================================================================
ERROR: test_paddlemd (__main__._TestPaddleMD) (system='test-data/4dihedrals/')
----------------------------------------------------------------------
Traceback (most recent call last):
  File "tests/test_paddlemd.py", line 408, in test_paddlemd
    system.pos, system.box, system.forces, returnDetails=True
  File "/code/6paper/PaddleMD/paddlemd/forces.py", line 284, in compute
    _, _, r12 = calculate_distances(
  File "/code/6paper/PaddleMD/paddlemd/forces.py", line 424, in calculate_distances
    #     print(f"==line 423 of forces direction_vec.shape:{direction_vec.shape}")
  File "/opt/conda/lib/python3.6/site-packages/paddle/tensor/linalg.py", line 436, in norm
    name=name)
  File "/opt/conda/lib/python3.6/site-packages/paddle/tensor/linalg.py", line 308, in vector_norm
    'keepdim', keepdim, 'asvector', asvector)
ValueError: (InvalidArgument) Attr(axis) value should be in range [-R, R-1], R is the rank of Input(X). But received axis: 1, R: 1. Current Input(X)'s shape is=[3].
  [Hint: Expected axis < x_rank, but received axis:1 >= x_rank:1.] (at /paddle/paddle/phi/infermeta/unary.cc:1488)
  [operator < p_norm > error]
```
## 先集中精力处理'Tensor' object has no attribute 'requires_grad'
```
======================================================================
ERROR: test_replicas (__main__._TestPaddleMD)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "tests/test_paddlemd.py", line 517, in test_replicas
    explicit_forces=False,
  File "/code/6paper/PaddleMD/paddlemd/forces.py", line 386, in compute
    if pot[i][ene].requires_grad:
AttributeError: 'Tensor' object has no attribute 'requires_grad'

----------------------------------------------------------------------
Ran 2 tests in 423.041s

FAILED (errors=7)
```
将requires_grad改成stop_gradient ，当然飞桨的语法是xx.stop_gradient=False
问题解决之后，出现新的报错
##   (PermissionDenied) Cannot set target var auto_1600864_@GRAD twice
说实话，这是我印象里头一次碰到这样的报错
```
======================================================================
ERROR: test_replicas (__main__._TestPaddleMD)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "tests/test_paddlemd.py", line 517, in test_replicas
    explicit_forces=False,
  File "/code/6paper/PaddleMD/paddlemd/forces.py", line 390, in compute
    enesum, pos, only_inputs=True, retain_graph=True
  File "</opt/conda/lib/python3.6/site-packages/decorator.py:decorator-gen-49>", line 2, in grad
  File "/opt/conda/lib/python3.6/site-packages/paddle/fluid/wrapped_decorator.py", line 25, in __impl__
    return wrapped_func(*args, **kwargs)
  File "/opt/conda/lib/python3.6/site-packages/paddle/fluid/framework.py", line 434, in __impl__
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.6/site-packages/paddle/fluid/dygraph/base.py", line 663, in grad
    retain_graph, allow_unused, only_inputs)
RuntimeError: (PermissionDenied) Cannot set target var auto_1600864_@GRAD twice
  [Hint: Expected ready_var.var == nullptr, but received ready_var.var != nullptr.] (at /paddle/paddle/fluid/imperative/partial_grad_engine.cc:508)
```
经了解，paddle.grad可能有些问题，不过我认为大概率还是自己的问题。 
这个报错，是说不能设置变量两次？ 

## 测试前丙氨酸二肽报错
```python
  File "/code/6paper/PaddleMD/bin/../paddlemd/run.py", line 154, in <module>
    mol, system, forces = setup(args)
  File "/code/6paper/PaddleMD/bin/../paddlemd/run.py", line 113, in setup
    forces = Forces(parameters, terms=args.forceterms, external=external, cutoff=args.cutoff, rfa=args.rfa, switch_dist=args.switch_dist, exclusions=args.exclusions)
  File "/code/6paper/PaddleMD/paddlemd/forces.py", line 103, in __init__
    'Set force terms or leave empty brackets [].\nAvailable options: "bonds", "angles", "dihedrals", "impropers", "1-4", "electrostatics", "lj", "repulsion", "repulsioncg".'
RuntimeError: Set force terms or leave empty brackets [].
Available options: "bonds", "angles", "dihedrals", "impropers", "1-4", "electrostatics", "lj", "repulsion", "repulsioncg".
```    
把forces.py文件中修改成：
```
if terms is None: # 为了不报错，我也是拼了
            terms = self.terms
```
终于可以跑下去了。

In [17]:
import numpy as np
a = np.random.randn(2,3,3)

# print(a)
import torch
ta = torch.tensor(a)
# print(ta)
ta = ta[:, torch.eye(3).bool()]
print(ta)
box = a
i = 0
sbox = box[i][torch.eye(3).bool()]
print(sbox)
sbox = box[:, torch.eye(3).bool()]
print(sbox)

tensor([[-0.3567,  0.9546, -0.2460],
        [ 0.1134,  0.3881, -0.4628]], dtype=torch.float64)
[-0.35666844  0.95460963 -0.24597028]
[[-0.35666844  0.95460963 -0.24597028]
 [ 0.11340753  0.38805964 -0.46283856]]


In [10]:
import paddle
box = paddle.to_tensor(a)
box = box*paddle.eye(3)
box = box.sum(1)
print(box)
box = paddle.to_tensor(a)
sbox = box[i][paddle.eye(3).astype(paddle.bool)]
print(sbox)
# box[:][paddle.eye(3).astype(paddle.bool)]
# print(box)

Tensor(shape=[2, 3], dtype=float64, place=Place(gpu:0), stop_gradient=True,
       [[ 0.78369857, -1.16356505,  0.76762212],
        [ 1.44171350, -0.14697054,  0.07857292]])
Tensor(shape=[3], dtype=float64, place=Place(gpu:0), stop_gradient=True,
       [ 0.78369857, -1.16356505,  0.76762212])


In [11]:
# box = box[:, torch.eye(3).bool()]
box = box*paddle.eye(3)
box = box.sum(1)
print(box)

Tensor(shape=[2, 3], dtype=float64, place=Place(gpu:0), stop_gradient=True,
       [[ 0.78369857, -1.16356505,  0.76762212],
        [ 1.44171350, -0.14697054,  0.07857292]])


In [7]:
sbox = box[i][torch.eye(3).bool()]

ValueError: (InvalidArgument) Currently, Tensor.__indices__() only allows indexing by Integers, Slices, Ellipsis, None, tuples of these types and list of Bool and Integers, but received Tensor in 1th slice item (at /paddle/paddle/fluid/pybind/slice_utils.h:279)


In [32]:
selfbox = ta
box = a
for r in range(box.shape[0]):
    selfbox[r][torch.eye(3).bool()] = torch.tensor(
        box[r], dtype=selfbox.dtype)
print(selfbox )            

IndexError: too many indices for tensor of dimension 1

In [33]:
selfbox[0][torch.eye(3).bool()] = torch.tensor(
        box[0], dtype=selfbox.dtype)

IndexError: too many indices for tensor of dimension 1

In [28]:
selfbox = paddle.to_tensor(a)
for r in range(box.shape[0]):
#             self.box[r][paddle.eye(3).astype(paddle.bool)] = paddle.to_tensor(
#                 box[r], dtype=self.box.dtype)
    selfbox[r] = paddle.to_tensor(
        box[r], dtype=selfbox.dtype) * paddle.eye(3).astype(paddle.bool)
print(selfbox)

Tensor(shape=[2, 3, 3], dtype=float64, place=Place(gpu:0), stop_gradient=True,
       [[[-0.35666844,  0.        ,  0.        ],
         [ 0.        ,  0.95460963,  0.        ],
         [ 0.        ,  0.        , -0.24597028]],

        [[ 0.11340753,  0.        ,  0.        ],
         [ 0.        ,  0.38805964,  0.        ],
         [ 0.        ,  0.        , -0.46283856]]])


In [34]:
# tmp = box*paddle.eye(3)

ValueError: (InvalidArgument) fill_constant(): argument (position 6) must be float, but got numpy.ndarray (at /paddle/paddle/fluid/pybind/op_function_common.cc:189)


## 飞桨grad例子


In [8]:
import paddle

def test_dygraph_grad(create_graph):
    x = paddle.ones(shape=[1,2], dtype='float32')
    x.stop_gradient = False
    y = x * x
    print(y)

    # Since y = x * x, dx = 2 * x
    dx = paddle.grad(
            outputs=[y],
            inputs=[x],
            create_graph=create_graph,
            retain_graph=True)[0]
    print(f"==dx:{dx}")
    z = y + dx

    # If create_graph = False, the gradient of dx
    # would not be backpropagated. Therefore,
    # z = x * x + dx, and x.gradient() = 2 * x = 2.0

    # If create_graph = True, the gradient of dx
    # would be backpropagated. Therefore,
    # z = x * x + dx = x * x + 2 * x, and
    # x.gradient() = 2 * x + 2 = 4.0

    z.backward()
    return x.gradient()

print(test_dygraph_grad(create_graph=False)) # [2.]
print(test_dygraph_grad(create_graph=True)) # [4.]


Tensor(shape=[1, 2], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[1., 1.]])
==dx:Tensor(shape=[1, 2], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [[2., 2.]])
[[2. 2.]]
Tensor(shape=[1, 2], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[1., 1.]])
==dx:Tensor(shape=[1, 2], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[2., 2.]])
[[4. 4.]]


In [9]:
import paddle

def test_dygraph_grad(grad_outputs=None):
    x = paddle.to_tensor(2.0)
    x.stop_gradient = False

    y1 = x * x
    y2 = x * 3

    # If grad_outputs=None, dy1 = [1], dy2 = [1].
    # If grad_outputs=[g1, g2], then:
    #    - dy1 = [1] if g1 is None else g1
    #    - dy2 = [1] if g2 is None else g2

    # Since y1 = x * x, dx = 2 * x * dy1.
    # Since y2 = x * 3, dx = 3 * dy2.
    # Therefore, the final result would be:
    # dx = 2 * x * dy1 + 3 * dy2 = 4 * dy1 + 3 * dy2.

    dx = paddle.grad(
        outputs=[y1, y2],
        inputs=[x],
        grad_outputs=grad_outputs)[0]

    return dx.numpy()

grad_value = paddle.to_tensor(4.0)

# dy1 = [1], dy2 = [1]
print(test_dygraph_grad(None)) # [7.]

# dy1 = [1], dy2 = [4]
print(test_dygraph_grad([None, grad_value])) # [16.]

# dy1 = [4], dy2 = [1]
print(test_dygraph_grad([grad_value, None])) # [19.]

# dy1 = [3], dy2 = [4]
grad_y1 = paddle.to_tensor(3.0)
print(test_dygraph_grad([grad_y1, grad_value])) # [24.]


[7.]
[16.]
[19.]
[24.]


In [15]:
class Forces:
    # 1-4 is nonbonded but we put it currently in bonded to not calculate all distances
    bonded = ["bonds", "angles", "dihedrals", "impropers", "1-4"]
    nonbonded = ["electrostatics", "lj", "repulsion", "repulsioncg"]
    terms = bonded + nonbonded
    def __init__(
        self,
        terms=None):
        print(f"terms:{terms}")
        if terms is None:
            terms = self.terms
        print(f"terms:{terms}")
test = Forces()

terms:None
terms:['bonds', 'angles', 'dihedrals', 'impropers', '1-4', 'electrostatics', 'lj', 'repulsion', 'repulsioncg']
